In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
import numpy as np
from datetime import date
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
from newsapi import NewsApiClient
import nltk
from statistics import mean 
from dotenv import load_dotenv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import requests
load_dotenv()
import os
from GoogleNews import GoogleNews
%matplotlib inline

In [5]:
alpaca_api_key = os.getenv("API_ALPACA_KEY")
alpaca_secret_key = os.getenv("API_ALPACA_SECRET_KEY")
type(alpaca_api_key)
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [6]:
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]

In [7]:
ticker = "TSLA"

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2018-07-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-07-05', tz='America/New_York').isoformat()

# Get 1 year's worth of historical data for TSLA
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

df.head()

TSLA                                    
                               open    high     low     close    volume
2018-07-02 00:00:00-04:00  360.2800  364.78  329.85  335.0700  14936701
2018-07-03 00:00:00-04:00  331.9100  332.49  309.69  310.9247   9605990
2018-07-05 00:00:00-04:00  313.7600  314.39  296.22  309.1100  13989015
2018-07-06 00:00:00-04:00  304.9545  312.07  302.00  308.9100   7097253
2018-07-09 00:00:00-04:00  311.8553  318.52  308.00  318.2900   5743049

In [8]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df.drop(columns=['open', 'high', 'low'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

df.head()

,close,volume
2018-07-02,335.0700,14936701
2018-07-03,310.9247,9605990
2018-07-05,309.1100,13989015
2018-07-06,308.9100,7097253
2018-07-09,318.2900,5743049


In [9]:
df['daily_returns_tsla']=df['close'].pct_change()
df['daily_returns_tsla_volume']=df['volume'].pct_change()
df.dropna(inplace=True)
df.drop(columns=['close', 'volume'], inplace=True)
df.head()

,daily_returns_tsla,daily_returns_tsla_volume
2018-07-03,-0.072060,-0.356887
2018-07-05,-0.005836,0.456280
2018-07-06,-0.000647,-0.492655
2018-07-09,0.030365,-0.190807
2018-07-10,0.012818,0.236055


In [10]:
y=[]
dr=df['daily_returns_tsla']
for i in dr:
  if i>0:
    y.append(1)
  else:
    y.append(0)
df['target']=y
df.head()

,daily_returns_tsla,daily_returns_tsla_volume,target
2018-07-03,-0.072060,-0.356887,0
2018-07-05,-0.005836,0.456280,0
2018-07-06,-0.000647,-0.492655,0
2018-07-09,0.030365,-0.190807,1
2018-07-10,0.012818,0.236055,1


In [11]:
ticker = "TLT"

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2018-07-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-07-05', tz='America/New_York').isoformat()


# Get 1 year's worth of historical data for TSLA
df_tlt = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

df_tlt.head()

TLT                                   
                             open    high       low   close   volume
2018-07-02 00:00:00-04:00  121.74  121.75  121.2901  121.36  8247467
2018-07-03 00:00:00-04:00  121.53  122.03  121.5300  121.98  3087020
2018-07-05 00:00:00-04:00  122.06  122.53  122.0100  122.33  3223311
2018-07-06 00:00:00-04:00  122.75  122.92  122.4800  122.71  3866259
2018-07-09 00:00:00-04:00  122.05  122.21  121.9000  121.96  4645923

In [12]:
df_tlt = df_tlt.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df_tlt.drop(columns=['open', 'high', 'low'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
df_tlt.index = df_tlt.index.date

df_tlt.head()

,close,volume
2018-07-02,121.36,8247467
2018-07-03,121.98,3087020
2018-07-05,122.33,3223311
2018-07-06,122.71,3866259
2018-07-09,121.96,4645923


In [13]:
df_tlt['daily_returns_tlt']=df_tlt['close'].pct_change()
df_tlt['daily_returns_tlt_volume']=df_tlt['volume'].pct_change()
df_tlt.drop(columns=['close', 'volume'], inplace=True)
df_tlt.dropna(inplace=True)
df_tlt.head()

,daily_returns_tlt,daily_returns_tlt_volume
2018-07-03,0.005109,-0.625701
2018-07-05,0.002869,0.044150
2018-07-06,0.003106,0.199468
2018-07-09,-0.006112,0.201659
2018-07-10,-0.000082,-0.079239


In [14]:
features_df=pd.concat([df_tlt,df],axis=1,join='inner')
features_df

,daily_returns_tlt,daily_returns_tlt_volume,daily_returns_tsla,daily_returns_tsla_volume,target
2018-07-03,0.005109,-0.625701,-0.072060,-0.356887,0
2018-07-05,0.002869,0.044150,-0.005836,0.456280,0
2018-07-06,0.003106,0.199468,-0.000647,-0.492655,0
2018-07-09,-0.006112,0.201659,0.030365,-0.190807,1
2018-07-10,-0.000082,-0.079239,0.012818,0.236055,1
...,...,...,...,...,...
2020-06-26,0.010827,0.203527,-0.025579,-0.116060,0
2020-06-29,-0.003433,-0.437685,0.051137,0.118197,1
2020-06-30,-0.004571,0.964535,0.068433,0.856565,1
2020-07-01,-0.003233,-0.134947,0.037572,-0.205057,1


In [15]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aljohara/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
dates=pd.DataFrame({'Date':pd.Series(features_df.index)})
dates.head()

,Date
0,2018-07-03
1,2018-07-05
2,2018-07-06
3,2018-07-09
4,2018-07-10


# Fetch news articles

In [17]:
articles_df = pd.DataFrame(columns=['date','text','source', 'neg', 'neu', 'pos', 'compound','stock_api_sent'])

In [123]:
# Read your api key environment variable

api_key = os.getenv("gnews")

In [23]:
# Create a newsapi client
#googlenews = GoogleNews(api_key)

In [20]:
#GNEWS:
import requests
import json

# Create parameterized url
request_url = "https://gnews.io/api/v3/search"#?q=example&token=API-Token"
query = 'tesla'




import time
n = len(dates)
for i in range(0,n,5):
    
    date = dates.loc[i]['Date']
    # minus 1 week
    date_begin = date - pd.to_timedelta(1, unit='w')
    #print(date,date_begin)

    # change it to MMDDYYYY formate for stocknewsapi call
    date = str(date)
    date_begin = str(date_begin)
    params = {"q":query,'token':api_key,'max':'100','mindate':date_begin,'maxdate':date}
    # Submit request and format output as json
    response_data = requests.get(request_url,params=params).json()
    count_articles = response_data['articleCount']
    try:
        returned_articles_gapi = response_data['articles']
    except:
        print('error', response_data)
    # add to articles_df
    for article in returned_articles_gapi:
        date = article['publishedAt']
        date = pd.to_datetime(date).date()
        text = article['title'] + ' '
        text += article['description']
        articles_df  = articles_df.append({'date':date,'text':text, 'source':'gapi'},ignore_index=True)
        
        time.sleep(1)

KeyError: 'articleCount'

In [24]:
# Read your api key environment variable
stocknewsapi_key = os.getenv("stock_news_api")

In [25]:
request_url = "https://stocknewsapi.com/api/v1"
query = 'TSLA'
params = {"tickers":query,'token':stocknewsapi_key,'items':'50','fallback':'true'}
# Submit request and format output
response_data = requests.get(request_url,params=params).json()


In [26]:
n = len(dates)
for i in range(n):
    date = dates.loc[i]['Date']
    # minus 1 week
    date_begin = date - pd.to_timedelta(1, unit='w')
    #print(date,date_begin)

    # change it to MMDDYYYY formate for stocknewsapi call
    date = str(date)
    date = date.split('-')
    date = date[1]+date[2]+date[0]
    date_begin = str(date_begin)
    date_begin = date_begin.split('-')
    date_begin = date_begin[1]+date_begin[2]+date_begin[0]
    
    date = date_begin+'-'+date
    #print(date)
    params = {"tickers":query,'token':stocknewsapi_key,'items':'50','fallback':'true','date':date}
    # Submit request and format output as json
    response_data = requests.get(request_url,params=params).json()
    articles = response_data['data']
    #print(response_data)
    #break
     # add to articles_df
    for article in articles:
        date = article['date']
        date = pd.to_datetime(date).date()
        text = article['title'] + ' '
        text += article['text']
        sentiment = article['sentiment']
        articles_df  = articles_df.append({'date':date,'text':text, 'source':'stocknews', 'stock_api_sent': sentiment},ignore_index=True)
        
        


In [27]:
# process sentiment of stocknewsapi
articles_df['stock_api_sent'] = articles_df['stock_api_sent'].fillna(0.5)
articles_df

,date,text,source,neg,neu,pos,compound,stock_api_sent
0,2018-07-02,Construction workers offer tips to stay cool T...,gapi,NaN,NaN,NaN,NaN,0.5
1,2018-07-01,NYT ed board: Time for new Dem leadership TRAD...,gapi,NaN,NaN,NaN,NaN,0.5
2,2018-06-30,"Elon Musk stole my dad’s farting unicorn art, ...",gapi,NaN,NaN,NaN,NaN,0.5
3,2018-06-29,Evergreen artist wants Elon Musk to puff up pa...,gapi,NaN,NaN,NaN,NaN,0.5
4,2018-06-29,Virgin Orbit Gears Up for Captive Carry Test F...,gapi,NaN,NaN,NaN,NaN,0.5
...,...,...,...,...,...,...,...,...
26193,2020-07-01,Elon Musk congratulates Tesla employees ahead ...,stocknews,NaN,NaN,NaN,NaN,Positive
26194,2020-07-01,Tesla hits record high—Here's what investors s...,stocknews,NaN,NaN,NaN,NaN,Positive
26195,2020-07-01,Tesla Owner Posts Video Claiming Autopilot Sav...,stocknews,NaN,NaN,NaN,NaN,Neutral
26196,2020-07-01,Tesla tops Toyota to become world’s most valua...,stocknews,NaN,NaN,NaN,NaN,Positive


In [28]:
n = len(articles_df)
for i in range(n):
    article = articles_df.iloc[i]
    #print(article)
    text = article['text']
    sentiment = analyzer.polarity_scores(text)    
    for sent in sentiment:
        articles_df.loc[i][sent] = sentiment[sent]

In [29]:
articles_df[articles_df['date']>pd.to_datetime('2018-06-26')]

,date,text,source,neg,neu,pos,compound,stock_api_sent
0,2018-07-02,Construction workers offer tips to stay cool T...,gapi,0,0.913,0.087,0.3818,0.5
1,2018-07-01,NYT ed board: Time for new Dem leadership TRAD...,gapi,0.104,0.83,0.066,-0.4019,0.5
2,2018-06-30,"Elon Musk stole my dad’s farting unicorn art, ...",gapi,0,0.884,0.116,0.7096,0.5
3,2018-06-29,Evergreen artist wants Elon Musk to puff up pa...,gapi,0.044,0.956,0,-0.296,0.5
4,2018-06-29,Virgin Orbit Gears Up for Captive Carry Test F...,gapi,0,0.897,0.103,0.5423,0.5
...,...,...,...,...,...,...,...,...
26193,2020-07-01,Elon Musk congratulates Tesla employees ahead ...,stocknews,0.075,0.65,0.276,0.8136,Positive
26194,2020-07-01,Tesla hits record high—Here's what investors s...,stocknews,0,0.898,0.102,0.5256,Positive
26195,2020-07-01,Tesla Owner Posts Video Claiming Autopilot Sav...,stocknews,0.141,0.778,0.081,-0.2732,Neutral
26196,2020-07-01,Tesla tops Toyota to become world’s most valua...,stocknews,0,0.8,0.2,0.9097,Positive


# Machine Learning Features

In [145]:
import datetime
new_features_df = features_df.copy()
new_features_df.drop(columns=['daily_returns_tlt','daily_returns_tlt_volume','daily_returns_tsla','daily_returns_tsla_volume'],inplace=True)
new_features_df.index = pd.to_datetime(new_features_df.index)

new_features_df = features_df.shift(1)
new_features_df['target'] = features_df['target']

i = 0
for date in new_features_df.index:
    start_date = date - pd.to_timedelta(3, unit='d')
    #print(start_date)
    #print(date)
    mask = (articles_df['date'] > start_date) & (articles_df['date'] <= date)
    #'''
    count_articles = articles_df.loc[mask].count()
    means = articles_df.loc[mask].mean()
    maxs = articles_df.loc[mask].max(axis=0)
    mins = articles_df.loc[mask].min(axis=0)
    
    features = {}

    for sent in ['neg','neu','pos','compound']:
        features['mean_'+sent] = means[sent]
        features['max_'+sent] = maxs[sent]
        features['min_'+sent] = mins[sent]
    
    for feature in features:
        new_features_df.loc[date,feature] = features[feature]
    # '''
    new_features_df.loc[date,'daily_returns_tsla'] = features_df.loc[date,'daily_returns_tsla']
    #new_features_df.iloc[i] = features_df.iloc[i-1].drop(columns='target')
    i += 1

In [146]:
new_features_df.loc[pd.to_datetime('2018-07-03').date(),'mean_neg'] = 1

In [147]:
new_features_df.drop(pd.to_datetime('2020-06-26'), inplace=True)
new_features_df.drop(pd.to_datetime('2020-06-29'), inplace=True)
new_features_df.drop(pd.to_datetime('2020-06-30'), inplace=True)
new_features_df.drop(pd.to_datetime('2020-07-01'), inplace=True)
new_features_df.drop(pd.to_datetime('2020-07-02'), inplace=True)

In [124]:
#new_features_df.dropna(inplace=True)

In [148]:
new_features_df

,daily_returns_tlt,daily_returns_tlt_volume,daily_returns_tsla,daily_returns_tsla_volume,target,mean_neg,max_neg,min_neg,mean_neu,max_neu,min_neu,mean_pos,max_pos,min_pos,mean_compound,max_compound,min_compound
2018-07-03,NaN,NaN,-0.072060,NaN,0,1.000000,0.155,0.0,0.870000,1.00,0.719,0.071067,0.125,0.0,-0.016740,0.3818,-0.4019
2018-07-05,0.005109,-0.625701,-0.005836,-0.356887,0,0.143583,0.158,0.0,0.801417,0.97,0.719,0.054583,0.125,0.0,-0.450450,0.0772,-0.6486
2018-07-06,0.002869,0.044150,-0.000647,0.456280,0,0.105333,0.158,0.0,0.891333,1.00,0.842,0.003333,0.030,0.0,-0.423822,0.0772,-0.6486
2018-07-09,0.003106,0.199468,0.030365,-0.492655,1,0.058259,0.212,0.0,0.878593,1.00,0.752,0.063148,0.168,0.0,0.122037,0.8176,-0.7506
2018-07-10,-0.006112,0.201659,0.012818,-0.190807,1,0.054905,0.212,0.0,0.893214,1.00,0.752,0.051881,0.168,0.0,0.097726,0.8176,-0.7506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19,0.010775,0.976910,-0.003436,1.063577,0,0.041610,0.344,0.0,0.883089,1.00,0.625,0.075342,0.375,0.0,0.201724,0.9217,-0.7845
2020-06-22,0.000616,-0.382424,-0.006027,-0.573621,0,0.018400,0.128,0.0,0.904436,1.00,0.566,0.077018,0.381,0.0,0.256433,0.9441,-0.8225
2020-06-23,0.000123,-0.071143,0.007159,-0.233255,1,0.039495,0.231,0.0,0.880682,1.00,0.566,0.079738,0.381,0.0,0.150993,0.9441,-0.8225
2020-06-24,-0.006572,0.125760,-0.040742,-0.012175,0,0.078699,0.277,0.0,0.852732,1.00,0.566,0.068614,0.381,0.0,-0.075773,0.9441,-0.8271


In [137]:
new_features_df['target'] = new_features_df['target'].astype(int)

In [128]:
y = new_features_df["target"]
X = new_features_df.drop(columns="target")

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
print(X_train.shape)
print(X_test.shape)

(373, 16)
(125, 16)


In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
classifier = SVC(C=1)
classifier.fit(X_train,y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [131]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))


Counter({1: 192, 0: 181})
Counter({1: 65, 0: 60})


In [132]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6729222520107239
Testing Data Score: 0.544


In [139]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test,
    "change": X_test['daily_returns_tsla']
}).reset_index(drop=True)
print(results)

In [143]:
results[results['Prediction']==results['Actual']]['change'].abs().mean()

In [141]:
results[results['Prediction']!=results['Actual']]['change'].abs().mean()